#### Unicode Fractions

In this notebook we test the processing of [Unicode](https://home.unicode.org) characters by playing around with [glyphs used for displaying fractions](https://en.wikipedia.org/wiki/Number_Forms).

First we create a *SAS* session and redefine `dbDISPLAY` - for details see the `Classy Cars` notebook.

In [0]:
import saspy
# Start a SAS Session using config file
sas = saspy.SASsession(
  cfgfile = '/databricks/sasconfig/sascfg_personal.py',
  cfgname = 'ssh_sas',
  display = 'databricks')
# Redefine `sas.DISPLAY`
def dbDISPLAY(x):
  displayHTML(x)
sas.DISPLAY = dbDISPLAY

SAS Connection established. Subprocess id is 1174

Then we extract the required fraction glyphs by parsing - here we utilize `curl` and `awk` on the command line with the `%sh` directive - the javascript library [math.js](https://github.com/lights0123/fractions/blob/master/math.js) from the [Unicode Fraction Creator](https://lights0123.com/fractions/) site, store the information in CSV format...

In [0]:
%sh
curl https://raw.githubusercontent.com/lights0123/fractions/master/math.js | \
awk -F'\x27' -v OFS=';' ' # \x27 is single quote
 BEGIN { print "ASCII;UTF8;TYPE" }
 /superscript/ {T="P"}
 /subscript/   {T="B"}
 /fractions/   {T="F"}
 NF>4 {print $2, $4, T}
' > /dbfs/FileStore/tables/sas_dbr_demo/ms.csv

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
100 3874 100 3874 0 0 32283 0 --:--:-- --:--:-- --:--:-- 32283

... and load it to the `ms` Pandas data frame. Note, that `TYPE='F'` contains the few fractions available as as a symbol. All other fractions have to be constructed from superscripts `TYPE='P'` and subscripts `TYPE='B'` together with the [FRACTION SLASH](https://util.unicode.org/UnicodeJsps/character.jsp?a=2044) character `⁄`.

In [0]:
import pandas as pd
ms = pd.read_csv("/dbfs/FileStore/tables/sas_dbr_demo/ms.csv", sep=';', encoding='utf-8')
display(ms)

ASCII,UTF8,TYPE
0,⁰,P
1,¹,P
2,²,P
3,³,P
4,⁴,P
5,⁵,P
6,⁶,P
7,⁷,P
8,⁸,P
9,⁹,P


Finally, create the Pandas data frame `uf` with all reduced fractions between 0 and 1 with denominator ≤12 ...

In [0]:
import math
D = 12
rs = [] # rational numbers 
fs = [] # fraction strings
ns = [] # numerators
ds = [] # denominators

def u8(ascii, type): # Receive UTF8 from `ms` given ASCII and TYPE  
  return ''.join( [ms.query(f"ASCII=='{c}' & TYPE=='{type}'").UTF8.iloc[0] for c in str(ascii)] )

for d in list(range(2,D+1)):
  for n in range(1,d):
    if math.gcd(d,n)==1:
      rs += [n/d]
      ds += [d]
      ns += [n]
      u = ms.query(f"ASCII=='{n}/{d}'")
      if not u.empty:
        fs += [u.UTF8.iloc[0]]
      else:
        fs += [f"{u8(n,'P')}⁄{u8(d,'B')}"] # NB. `⁄` is FRACTION SLASH

uf = pd.DataFrame({ 'r': rs, 'f': fs, 'n': ns, 'd': ds })
display(uf)


r,f,n,d
0.5,½,1,2
0.3333333333333333,⅓,1,3
0.6666666666666666,⅔,2,3
0.25,¼,1,4
0.75,¾,3,4
0.2,⅕,1,5
0.4,⅖,2,5
0.6,⅗,3,5
0.8,⅘,4,5
0.16666666666666666,⅙,1,6


... after transfering the data frame to *SAS* table `uf` we create a *SQL* query in *SAS*. Note, that we have to use [National Language Support (NLS) functions](https://documentation.sas.com/doc/en/pgmsascdc/9.4_3.5/nlsref/p18bboh5zrwqw5n1kkhonig4jpwq.htm) like `unicodelen` and `kindex` (column `xs` is the index of the FRACTION SLASH); `length` will mislead us here as it shows the byte length `bl` which for example is different for `½` and `⅓` depending on their *UTF-8* representation.

In [0]:
uf_sas = sas.df2sd( uf, 'uf' )
sas.submitLST("""
 title "Reduced fractions with denominator ≤12";
 proc sql; select *, length(f) AS bl, unicodelen(f) AS ul, kindex(f,'⁄') AS xs from uf; quit;
""")


r,f,n,d,bl,ul,xs
0.5,½,1,2,2,1,0
0.333333,⅓,1,3,3,1,0
0.666667,⅔,2,3,3,1,0
0.25,¼,1,4,2,1,0
0.75,¾,3,4,2,1,0
0.2,⅕,1,5,3,1,0
0.4,⅖,2,5,3,1,0
0.6,⅗,3,5,3,1,0
0.8,⅘,4,5,3,1,0
0.166667,⅙,1,6,3,1,0


Showing all our reduced fractions in a `PROC SGPLOT` bubble plot.
Only *SVG* output format renders all fractions correctly, if using any image format like *PNG* or *JPG* the 3 fractions `⅐`, `⅑` and `⅒` cannot be displayed. Reason might be, that they were not part of Unicode 1.0 - see [Searching in 𝕌𝕟𝕚𝕔𝕠𝕕𝕖 character names for VULGAR FRACTION](https://unicode-search.net/unicode-namesearch.pl?term=VULGAR%20FRACTION) - and *SAS* missed an update...

In [0]:
sas.submitLST('''
ods html5 (id=saspy_internal) file=stdout options(svg_mode='inline') device=svg style=Dove;
ods graphics on / outputfmt=svg;
proc sgplot data=uf noborder;
  bubble x=d y=n size=r /
    fillattrs=(color=turqois transparency=.3) bradiusmin=3px bradiusmax=12px nooutline
    datalabel=f datalabelpos=right datalabelattrs=(family="Arial Unicode MS" size=12);
  xaxis values=(2 to 12 by 1) grid display=(noline noticks nolabel);
  yaxis values=(1 to 11 by 1) grid display=(noline noticks nolabel);
run;
''')
# Arial Unicode MS
# Times New Roman Uni

<!DOCTYPE html>
 
 
 
 
 SAS Output 

 
 
 
 
<!DOCTYPE html>
 
 
 
 
 SAS Output 

 
 
 
 
 

 
 svgtitle 
 The SGPlot Procedure 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ¹¹⁄₁₂ 
 
 
 ¹⁰⁄₁₁ 
 
 
 ⁹⁄₁₀ 
 
 
 ⁸⁄₉ 
 
 
 ⅞ 
 
 
 ⁶⁄₇ 
 
 
 ⅚ 
 
 
 ⁹⁄₁₁ 
 
 
 ⅘ 
 
 
 ⁷⁄₉ 
 
 
 ¾ 
 
 
 ⁸⁄₁₁ 
 
 
 ⁵⁄₇ 
 
 
 ⁷⁄₁₀ 
 
 
 ⅔ 
 
 
 ⁷⁄₁₁ 
 
 
 ⅝ 
 
 
 ⅗ 
 
 
 ⁷⁄₁₂ 
 
 
 ⁴⁄₇ 
 
 
 ⁵⁄₉ 
 
 
 ⁶⁄₁₁ 
 
 
 ½ 
 
 
 ⁵⁄₁₁ 
 
 
 ⁴⁄₉ 
 
 
 ³⁄₇ 
 
 
 ⁵⁄₁₂ 
 
 
 ⅖ 
 
 
 ⅜ 
 
 
 ⁴⁄₁₁ 
 
 
 ⅓ 
 
 
 ³⁄₁₀ 
 
 
 ²⁄₇ 
 
 
 ³⁄₁₁ 
 
 
 ¼ 
 
 
 ²⁄₉ 
 
 
 ⅕ 
 
 
 ²⁄₁₁ 
 
 
 ⅙ 
 
 
 ⅐ 
 
 
 ⅛ 
 
 
 ⅑ 
 
 
 ⅒ 
 
 
 ¹⁄₁₁ 
 
 
 ¹⁄₁₂ 
 
 
 
 
 
 
 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 8 
 
 
 9 
 
 
 10 
 
 
 11 
 
 
 12 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 8 
 
 
 9 
 
 
 10 
 
 
 11 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Reduced fractions with denominator ≤12 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 The SGPlot Procedure 
 
 
 
 
 <![CDATA[ ]]> 
 
 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<glyph unicode="⁸" horiz-adv-x="680" 
d="M465 1090Q552 1070 596 1030 T640 929Q640 837 557 781 T337 725Q202 725 122 
781 T42 929Q42 990 83 1031 T207 1090Q146 1113 115 1156 T85 1261Q85 1346 
150 1398 T334 1450Q458 1450 527 1398 T597 1256Q597 1196 564 1155 T465 1090ZM340 
1152Q378 1152 402 1175 T426 1238Q426 1273 401 1298 T338 1322Q302 1322 279 1298 T
256 1240Q256 1199 279 1175 T340 1152ZM339 853Q398 853 433 877 T469 938Q469 
975 433 999 T337 1024Q280 1024 246 1000 T213 940Q213 900 246 876 T339 853Z" > 
 
<glyph unicode="⅞" horiz-adv-x="1700" 
d="M725 1365Q594 1200 544 1049 T464 725L293 725Q304 881 355 1035 T554 1322L170 
1322L170 1450L725 1450L725 1365ZM1493 1493L414 -42L213 -42L1292 1493L1493 1493ZM
1489 322Q1576 302 1620 262 T1664 161Q1664 69 1581 13 T1361 -42Q1226 -42 1146 
13 T1066 161Q1066 222 1107 263 T1231 322Q1170 345 1139 388 T1109 493Q1109 578 
1174 630 T1358 682Q1482 682 1551 630 T1621 488Q1621 428 1588 387 T1489 322ZM1364 
384Q1402 384 1426 407 T1450 470Q1450 505 1425 530 T1362 554Q1326 554 1303 530 T
1280 472Q1280 431 1303 407 T1364 384ZM1363 85Q1422 85 1457 109 T1493 170Q1493 
207 1457 231 T1361 256Q1304 256 1271 232 T1237 172Q1237 132 1271 108 T1363 85Z" > 
 
 
<glyph unicode="⅚" horiz-adv-x="1700" 
d="M226 1192Q302 1237 384 1237 Q491 1237 565 1164 T640 990Q640 875 549 800 T325 
725Q210 725 133 776 T32 922L202 938Q215 896 246 874 T326 853Q390 853 
429 889 T469 983Q469 1034 431 1071 T335 1109Q276 1109 216 1066 L74 1091L154 
1450L597 1450L597 1322L255 1322L226 1192ZM1493 1493L414 -42L213 -42L1292 1493L
1493 1493ZM1237 325Q1310 426 1428 426 Q1528 426 1596 361 T1664 201Q1664 100 1583 
28 T1377 -42Q1231 -42 1149 49 T1066 300Q1066 478 1159 580 T1417 682Q1522 682 
1587 644 T1674 530L1504 512Q1483 554 1398 554 Q1319 554 1279 499 T1237 325ZM1384 
85Q1434 85 1463 115 T1493 192Q1493 238 1465 268 T1392 298Q1344 298 1312 266 T
1280 182Q1280 140 1308 112 T1384 85Z" > 
 
 
<glyph unicode="¾" horiz-adv-x="1700" 
d="M1546 1493L374 0L245 0L1417 1493L1546 1493ZM457 1097Q640 1058 640 910 Q640 
816 562 749 T344 682Q214 682 138 732 T42 882L213 896Q227 852 258 831 T
339 810Q395 810 432 844 T469 928Q469 1026 323 1026 L256 1024L275 1152Q341 
1152 384 1175 T426 1243Q426 1276 402 1299 T328 1322Q284 1322 255 1301 T213 1237L
42 1259Q81 1367 146 1409 T325 1450Q452 1450 525 1402 T597 1262Q597 1140 457 
1101 ZM1578 85L149